In [3]:
import pandas as pd

In [25]:
df = pd.read_csv('Cleaned-data.csv')
df.head()

,Unnamed: 0,id,Phrase,Sentiment
0,0,0,کیفیت حجم صدای محصول توی بازار اصلا نمیشه تهرا...,100
1,1,1,ماه مصرف دوبار مدل گرفتم اولاش خوبه ماه باد می...,5
2,2,2,کارآیی کارای سبک دیدن فیلم مطالعه مناسبه,60
3,3,3,بررسی کیفی خریدش شگفت انگیز حتما پیشنهاد,0
4,4,4,بسته ضعیف ظاهر بامزه ای داره عکسش شبیه جنس چوب...,60


In [39]:
seq_len = 512
num_samples = len(df)

num_samples, seq_len

(179348, 512)

In [6]:
df['Phrase'] = df['Phrase'].astype('str')

In [9]:
!pip install tensorflow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 KB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 23.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 153.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 24.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 24.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 170.7 MB/

In [10]:
from transformers import AutoTokenizer

# initialize tokenizer
model_name = "HooshvareLab/bert-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenize - this time returning Numpy tensors
tokens = tokenizer(df['Phrase'].tolist(), max_length=seq_len, truncation=True,
                   padding='max_length', add_special_tokens=True,
                   return_tensors='np')

In [11]:
tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [12]:
tokens['input_ids'][:10]

array([[    2,  3114,  4035, ...,     0,     0,     0],
       [    2,  2208,  2763, ...,     0,     0,     0],
       [    2,     1, 22195, ...,     0,     0,     0],
       ...,
       [    2,  2915,  7060, ...,     0,     0,     0],
       [    2,  3114,  2297, ...,     0,     0,     0],
       [    2,  2395, 12228, ...,     0,     0,     0]])

In [13]:
tokens['attention_mask'][:10]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [15]:
import numpy as np

with open('movie-xids.npy', 'wb') as f:
    np.save(f, tokens['input_ids'])
with open('movie-xmask.npy', 'wb') as f:
    np.save(f, tokens['attention_mask'])

In [34]:
df['Sentiment'] = df['Sentiment'].div(20).round(0)
df['Sentiment']

0         5.0
1         0.0
2         3.0
3         0.0
4         3.0
         ... 
179343    5.0
179344    3.0
179345    3.0
179346    0.0
179347    3.0
Name: Sentiment, Length: 179348, dtype: float64

In [44]:
# first extract sentiment column
arr = df['Sentiment'].values
arr = arr.astype('int')

In [46]:
# we then initialize the zero array
labels = np.zeros((num_samples, arr.max()+1))
labels.shape

(179348, 6)

In [47]:
labels[np.arange(num_samples), arr] = 1

labels

array([[0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [48]:
with open('movie-labels.npy', 'wb') as f:
    np.save(f, labels)